In [3]:
# Scraping libraries
import pandas as pd
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup

import time

# Google sheet libraries
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

In [5]:
driver = webdriver.Chrome(executable_path=r"C:\Users\Matthew\Desktop\Double_io\Mortgage Match\chromedriver.exe")
driver.get('https://mortgagerates.co.nz/mortgage-rates')

time.sleep(3)

In [14]:
headers_path = driver.find_elements_by_xpath('/html/body/main/section[2]/div/div[2]/div/div/div/div[2]/div[1]')
headers = [item.text for item in headers_path[0].find_elements_by_xpath(".//*/div[@class='table__data']")][1:]
interest_rates = []
html = driver.find_element_by_xpath("//*").get_attribute("outerHTML")
soup = BeautifulSoup(html, 'html.parser')
body = soup.find_all('span', class_='product-name')
names = [item.text for item in body]
for table in driver.find_elements_by_xpath('/html/body/main/section[2]/div/div[2]/div/div/div/div[2]/div[2]/div'):
    data = [item.text for item in table.find_elements_by_xpath(".//*[self::div[@class='table__data']]")]
    interest_rates.append(data)

In [31]:
dftest = pd.DataFrame(interest_rates, columns=headers)
dftest.insert(loc=0, column='Bank/Lenders', value=names)

dftest


,Bank/Lenders,Revolving,Floating,6 Months,1 Year,18 Months,2 Years,3 Years,4 Years,5 Years
0,ANZ,4.55%,4.44%,3.99%,2.89%,3.15%,3.25%,3.39%,3.90%,3.99%
1,ANZ Special,,,3.39%,2.29%,2.55%,2.69%,2.79%,,
2,ASB,4.55%,4.45%,3.39%,2.29%,2.49%,2.59%,2.65%,2.99%,2.99%
3,BNZ Owner Occupied Standard,,4.55%,3.99%,2.89%,3.09%,3.19%,3.39%,3.59%,3.59%
4,BNZ Owner Occupied Special,,,3.39%,2.29%,2.49%,2.59%,2.79%,2.99%,2.99%
5,BNZ Investor Standard,,4.55%,3.99%,2.89%,3.09%,3.19%,3.39%,3.59%,3.59%
6,BNZ Investor Special,,,3.39%,2.29%,2.49%,2.59%,2.79%,2.99%,2.99%
7,China Construction Bank,,5.00%,4.70%,4.70%,,4.80%,4.95%,4.95%,4.95%
8,China Construction Bank Special,,,,2.65%,2.65%,2.65%,2.80%,2.89%,2.99%
9,Credit Union Auckland,,5.95%,,,,,,,


In [28]:
# Configure the connection 
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

# Give the path to the Service Account Credential json file 
credentials = ServiceAccountCredentials.from_json_keyfile_name('creds.json',scope)

# Authorise your Notebook
gc = gspread.authorize(credentials)

# The spread sheet ID, which can be taken from the link to the sheet
spreadsheet_key = '15mwMnp7QPr-XbNiFFQJcc-txxPVQhB_RKY_TphLW1T8'

In [34]:
d2g.upload(dftest,
           spreadsheet_key,
           "Master",
           credentials=credentials,
          row_names=False)

<Worksheet 'Master' id:1230338845>

In [7]:
#################---------------------------------------#####################